# Deep Learning Project

TODO Add names
pip install libretranslate
then run
libretranslate



In [13]:
%pip install libretranslatepy

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.utils as utils
import torchvision.transforms as transforms
import requests
from torch.utils.data import DataLoader

Note: you may need to restart the kernel to use updated packages.


**Run libretranslate on http://127.0.0.1:5000**

*Add the languages to translate in --load-only <comma-separated language codes> Set available languages (ar,de,en,es,fr,ga,hi,it,ja,ko,pt,ru,zh)*

**Jupyter does not allow the process to runn in the background, thus start a console and run libretranslate --load-only en,de**

**Add the Dataset**

In [ ]:
from torchvision import transforms
transforms_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Loading the CIFAR-10 dataset:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_cifar)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_cifar)



Using downloaded and verified file: ./data/cifar-10-python.tar.gz
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


*Iterate through the dataset and translate, specifiy the source and target language. For batch processing, add multiple requests into the array*

In [19]:
url = "http://127.0.0.1:5000/translate"

params = {"q" : ["Hello, how are you?", "What is your name?"],
          "source" : "en",
          "target" : "de"
}

response = requests.post(url, json=params)

if response.status_code == 200:
    translated_text = response.json()["translatedText"]
    print(translated_text)
else:
    print(f"Request failed with status code {response.status_code}")

["Hallo, wie geht's?", 'Wie heißt du?']


**Load the pretrained Model**

In [17]:
# MIT Copyright (c) 2019 Huy Phan. Model taken from https://github.com/huyvnphan/PyTorch_CIFAR10/

# Note: make sure to add the 'mobilenet_v2.pt' and 'mobilenetv2.py' files into the current working directory
from mobilenetv2 import mobilenet_v2

# Pretrained teacher model
teacher_model = mobilenet_v2()
get_model_inference_time(teacher_model)
teacher_model_weights_path = 'mobilenet_v2.pt'
teacher_model.load_state_dict(torch.load(teacher_model_weights_path))
teacher_model.to(device)

print('model size', get_model_size(teacher_model_weights_path))

print('teacher performance:')
test_loss, test_metrics = evaluate(teacher_model, criterion, metrics, test_loader, device)

from torchsummary import summary
summary(teacher_model, (3, 32, 32))
#accuracy of a teacher model should be around 0.93


model inference time in seconds: 3.3402442932128906
model weights file size in MB: 8.767388343811035
model size 8.767388343811035
teacher performance:
eval Loss: 0.2463,  ACC: 0.9355, F1-weighted: 0.9315
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
             ReLU6-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 32, 32, 32]             288
       BatchNorm2d-5           [-1, 32, 32, 32]              64
             ReLU6-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 16, 32, 32]             512
       BatchNorm2d-8           [-1, 16, 32, 32]              32
  InvertedResidual-9           [-1, 16, 32, 32]               0
           Conv2d-10           [-1, 96, 32, 32]           1,536
      BatchNorm2d-11       